In [56]:
%matplotlib widget

In [57]:
import numpy

In [58]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset

In [59]:
from skopt import Optimizer as skoptim

In [60]:
from matplotlib import pyplot as plot

In [61]:
xs = torch.tensor([[0,0], [1,1], [1,-1], [-1,1], [-1,-1]], dtype=torch.float32)
ys = torch.tensor([1, 0, 0, 0, 0], dtype=torch.float32)
trainset = TensorDataset(xs, ys)
trainloader = DataLoader(trainset, batch_size=1, shuffle=True)

In [62]:
def push(param, model):
    param = numpy.array(param)
    idx = 0
    for pp in model.parameters():
        n_pp = numpy.array(pp.shape).prod()
        pp.data.copy_(torch.from_numpy(param[idx:idx+n_pp].reshape(pp.shape)))
        idx = idx + n_pp
    return model

In [63]:
model = nn.Sequential(nn.Linear(2,16), nn.Tanh(), 
#                       nn.Linear(16,4), nn.Tanh(), 
                      nn.Linear(16,1), nn.Sigmoid())

In [64]:
parameter_shapes = [pp.shape for pp in model.parameters()]
n_parameters = numpy.sum([numpy.array(sh).prod() for sh in parameter_shapes])

In [65]:
optimizer = skoptim([(-10.,10.)] * n_parameters, base_estimator="GBRT")

In [66]:
criterion = nn.BCELoss()

In [67]:
epochs = 100
disp_interval = 10

run_avg = -numpy.Inf

for epoch in range(epochs):
    suggested = optimizer.ask()
    model = push(suggested, model)    
    loss = criterion(model(xs).view(-1), ys)
    res = optimizer.tell(suggested, loss.item())
    
    if run_avg == -numpy.Inf:
        run_avg = loss.item()
    else:
        run_avg = 0.9 * run_avg + 0.1 * loss.item()
    
    if (epoch+1) % 10 == 0:
        best_index = numpy.argmin(res.func_vals)
        print('iteration:', epoch+1, run_avg, res.func_vals[best_index])
        torch.save(model.state_dict(), F'./ckpt/skopt_epoch_{epoch+1}.ckpt')

iteration: 10 28.649975632189328
iteration: 20 21.096564846638138
iteration: 30 24.946689547785283
iteration: 40 21.38251710292008
iteration: 50 21.935046229467083
iteration: 60 23.778768532585993
iteration: 70 25.551611038046843
iteration: 80 22.00644319055988
iteration: 90 12.89369944401147
iteration: 100 19.12203105650732
iteration: 110 20.666597513385923
iteration: 120 19.09175006709904
iteration: 130 19.755899093885937
iteration: 140 18.021262053695054
iteration: 150 18.620276178433155
iteration: 160 18.779684387750464
iteration: 170 22.573745403328896
iteration: 180 23.073068577817494
iteration: 190 24.59616545939477
iteration: 200 17.310118224775962
iteration: 210 18.516568018617825
iteration: 220 21.161142878263153
iteration: 230 13.536195257996186


KeyboardInterrupt: 

In [68]:
res = optimizer.get_result()

In [69]:
best_index = numpy.argmin(res.func_vals)

In [70]:
loss = res.func_vals[best_index]
param = res.x_iters[best_index]
model = push(param, model)

In [71]:
X1p = numpy.arange(-2,2,0.1, dtype='float32')
X2p = numpy.arange(-2,2,0.1, dtype='float32')
X1, X2 = numpy.meshgrid(X1p, X2p)

out = model(torch.tensor(numpy.vstack([X1.flatten(), X2.flatten()]).T))

In [72]:
plot.figure()

plot.contour(X1, X2, out.data.numpy().reshape(X1.shape))

for x, y in zip(xs, ys):
    if y == 0:
        plot.plot(x[0], x[1], 'ro')
    else:
        plot.plot(x[0], x[1], 'bx')
        
plot.title(F'loss {loss}')

plot.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …